<a href="https://colab.research.google.com/github/kevinaldyansyah/SPK-with-python/blob/main/TOPSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Library Numpy dan disimapan dalam nama 'np'
import numpy as np

# Membuat array 2d sesuai dengan matrix penyekalaan
init_matrix = np.array([[3,4,5], [5,3,4], [3,2,5], [1,3,1]])
# Cek hasil array 2d
print(init_matrix)

[[3 4 5]
 [5 3 4]
 [3 2 5]
 [1 3 1]]


In [2]:
# Pada modul ini akan diterapkan normalisasi matrix sesuai dengan persamaan
# Kita akan membuat fungsi yang akan menerima paramater berupa array 2d
import math

def normalization(matrix):
    row_values = []
    norm_matrix = []
    
    for i in range(matrix.shape[0]): # Looping per baris (kriteria)
        # Menghitung sum tiap x_{ij}^2
        sum_row = sum([pow(x,2) for x in matrix[i]])
        
        for j in range(matrix[i].shape[0]): # Looping per kolom (alternatif)
            # membangi nilai asli x_{ij} dengan hasil akar
            r_value = matrix[i][j] / math.sqrt(sum_row)
            
            # Masukkan hasil normalisasi ke list tiap baris
            row_values.append(r_value)
        
        #Masukkan hasil normalisasi per baris ke matrix normalisasi
        norm_matrix.append(row_values)
        
        #Kosongkan list normalisasi perbaris
        row_values = []
            
    # Rturn matrix normalisasi dalam bentuk numpy array
    return np.asarray(norm_matrix)

In [3]:
# Testing Fungsi Normalisasi
norm = normalization(init_matrix)
print(norm)

[[0.42426407 0.56568542 0.70710678]
 [0.70710678 0.42426407 0.56568542]
 [0.48666426 0.32444284 0.81110711]
 [0.30151134 0.90453403 0.30151134]]


In [4]:
# Kalkulasi skor normalisasi terbobot

# Bobot untuk Kriteria C1, C2, C3, dan C4
c_weights = np.array([30,25,20,25])

# Fungsi untuk kalkulasi matrix terbobot. Paramter yang diperlukan adalah nilai ternormalisasi dan bobot
# Ingat! Kriteria adalah baris, Kolom adalah alternatif
def weighted_normalization(n_matrix, c_weights):
    # Buat salinan nilai ternormalisasi
    norm_weighted = n_matrix
    
    for i in range(c_weights.shape[0]): # Looping tiap kriteria
        # Kalkulasi normalisasi terbobot
        norm_weighted[i] = [r * c_weights[i] for r in norm_weighted[i]]
    
    return np.asarray(norm_weighted)

In [5]:
# Testing Matrix Normalisasi terbobot
w_norm = weighted_normalization(norm, c_weights)
print(w_norm)

[[12.72792206 16.97056275 21.21320344]
 [17.67766953 10.60660172 14.14213562]
 [ 9.73328527  6.48885685 16.22214211]
 [ 7.53778361 22.61335084  7.53778361]]


In [6]:
# Membuat label benefit dan cost untuk tiap kriteria
# Benefit = 1
# Cost = 0
c_label = np.array([1, 1, 1, 0])

In [7]:
# Fungsi mencari solusi ideal positif dan negatif
# Parameter yang dibutuhkan adalah, matrix normalisasi terbobot dan label benefit cost untuk tiap kriteria

def ideal(w_norm, c_label):
    a_positif = []
    a_negatif = []
    
    for i in range(w_norm.shape[0]):
        if c_label[i] == 1:
            # Untuk ideal positif
            a_max = max(w_norm[i])
            a_positif.append(a_max)
            
            # Untuk ideal negatif
            a_min = min(w_norm[i])
            a_negatif.append(a_min)
        elif c_label[i] == 0:
            # Untuk ideal positif
            a_max = min(w_norm[i])
            a_positif.append(a_max)
            
            # Untuk ideal negatif
            a_min = max(w_norm[i])
            a_negatif.append(a_min)
    
    ideal_value = np.array([a_positif, a_negatif])
    
    # Return dalam bentuk transpose sehingga baris = kriteria, kolom = solusi ideal positif dan negatid
    return ideal_value.transpose()

In [8]:
# Testing Solusi Ideal
ideal_v = ideal(w_norm, c_label)
print(ideal_v)

[[21.21320344 12.72792206]
 [17.67766953 10.60660172]
 [16.22214211  6.48885685]
 [ 7.53778361 22.61335084]]


In [9]:
# Kalkulasi Jarak Ideal Positif dan Negatif
# Disini akan menggunakan bantuan fungsi `distance` dari library `scipy.spatial`
# Parameter :
# 1. Matrix normalisasi terbobot
# 2. Nilai solusi ideal positif dan negatif

from scipy.spatial import distance as d

def alt_ideal_distance(w_norm, ideal_v):
    d_positif = []
    d_negatif = []
    
    # Kalkulasi Jarak
    for i in range(w_norm[0].shape[0]):
        # positif
        dp = d.euclidean(w_norm[:,i], ideal_v[:,0])
        d_positif.append(dp)
        
        # negatif
        dn = d.euclidean(ideal_v[:,1], w_norm[:,i])
        d_negatif.append(dn)
    
    d_positif = np.asarray(d_positif)
    d_negatif = np.asarray(d_negatif)
    
    d_value = np.array([d_positif, d_negatif])
        
    return d_value.transpose()

In [10]:
# Testing Jarak Ideal Positif
distance = alt_ideal_distance(w_norm, ideal_v)
print(distance)

[[10.68200651 16.96464096]
 [19.74865994  4.24264069]
 [ 3.53553391 20.16208247]]


In [11]:
distance.shape

(3, 2)

In [12]:
# Kalkulsi skor akhir
# Parameter :
# 1. distance : matrix jarak solusi positif dan negatif. 
# Baris adalah kriteria, kolom adalah solusi ideal positif dan negatif

def final_rank(distance):
    v = []
    
    for i in range(distance.shape[0]):
        vi = distance[i][1] / (distance[i][1] + distance[i][0])
        v.append(vi)
    
    return np.asarray(v)

In [13]:
# Testing skor akhir
ranking = final_rank(distance)
print(ranking)

[0.6136238  0.1768408  0.85080635]
